### final

In [1]:
## datadam_sales_per_area

## 확인사항
## 1. KT데이터 업데이트 여부
## 2. 건축물대장 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
date = '202210'
# dates = pd.date_range('2022-07',periods=3,freq='M').strftime("%Y%m")

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# INSERT
aws_cursor.execute(
"""
insert into m2.datadam_sales_per_area (
with date as (
    select '""" + date + """'::text as base_ym
)
select (select base_ym from date) as base_ym
     , T1.pnu
     , substring(T1.pnu, 1, 8) as emd_cd 
     , T1.industry_cd
     , round(1.0 * T1.sales / 33, 0) as sales
     , sysdate::date as create_at
     , sysdate::date as update_at
     , 'du.Park' as work_user
from (
    select substring(bd_mgt_sn, 1, 19) as pnu
         , ftc_cate2_cd as industry_cd
         , avg(pred_slng_amt) as sales
    from m1.kt_bldg_sales 
    where 1=1
        and base_ym = (select base_ym from date) -- 기준월
    group by 1, 2
) T1
join m2.cremao_land T2
    on T1.pnu = T2.pnu
join (
    select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
    from m1.bld_rgst
    where 1=1
        and base_ym = (select base_ym from date) -- 기준월
) T3
    on T1.pnu = T3.pnu
);
"""
)
aws_conn.commit()
print("Done")

Done


###### ver1

In [19]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 법정동별 상가 면적
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 매출액
sql = '''
select lgl_dong_cd||'00' as emd_cd, avg(avg_biz_income) as sales_avg
  from m1.kcb_soho	 
 group by lgl_dong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kcb_soho = pd.DataFrame(rows)
df_kcb_soho.columns = ['emd_cd','sales_avg']
df_kcb_soho['sales_avg'] = (df_kcb_soho['sales_avg']*1000).astype(float)


# 데이터 프레임 병합
df = pd.merge(df_kcb_soho, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 매출액 / 면적
df['sales_per_area'] = (df['sales_avg'] / df['total_count'])
df = df[['emd_cd','sales_per_area']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_sales_per_area;
    create table m2.datadam_sales_per_area (    
    emd_cd varchar(10)
    , sales_per_area double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 20:29:02.684279  :  0
3650


In [15]:
df

,emd_cd,sales_avg,total_count
0,1111010400,68462000.000000,39.000000
1,1111010700,148435000.000000,42.000000
2,1111010800,64937000.000000,115.000000
3,1111010900,43201000.000000,17.000000
4,1111011000,107058000.000000,78.000000
...,...,...,...
4934,4617012900,19364000.000000,46.000000
4935,4219011000,14371000.000000,15.000000
4936,4518011100,84256000.000000,60.000000
4937,3611011700,36791000.000000,19.000000


In [18]:
df.describe()

,sales_per_area
count,3650.000000
mean,3459872.963816
std,15581744.463154
min,0.000000
25%,238977.037037
50%,494613.013699
75%,1257581.140351
max,439616000.000000


###### ver2 (kt 업종 데이터 가져와서 병합하면 메모리터짐)

In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 평균 연면적
sql = '''
select sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu, avg(tot_area) as avg_area, substring(pnu,0,11)
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
 group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['pnu','area_avg','emd_cd']

# PNU별 매출액
sql = '''
select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00'
  from m1.kt_bldg_sales
 where base_ym = '202209'
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kt_bldg_sales = pd.DataFrame(rows)
df_kt_bldg_sales.columns = ['pnu','sales','industry_cd','emd_cd']

In [6]:
df_bld_rgst

,pnu,area_avg,emd_cd
0,1111010100100010000,449.8300,1111010100
1,1111010100100050000,397.6000,1111010100
2,1111010100100070003,81.9000,1111010100
3,1111010100100300007,136.3400,1111010100
4,1111010100100300013,998.3000,1111010100
...,...,...,...
2077137,4211040028106890012,60.0000,4211040028
2077138,4885039026202880000,78.0000,4885039026
2077139,4888025027111860001,47.7900,4888025027
2077140,4613015000108920000,66.9600,4613015000


In [7]:
df_kt_bldg_sales

,pnu,sales,industry_cd,emd_cd
0,3114010100102820002,5740478,B14,3114010100
1,3114010100102820002,100000000,B21,3114010100
2,3114010100102820002,1323405,B20,3114010100
3,3114010100102820002,12688502,A07,3114010100
4,3114010100102820002,8821770,A02,3114010100
...,...,...,...,...
58555804,5013032026117900000,14151894,C01,5013032000
58555805,5013032026117900000,8786831,B10,5013032000
58555806,5013032026117900000,2477928,A05,5013032000
58555807,5013032026117900000,26818739,A10,5013032000


In [12]:
# 데이터 프레임 병합
df_ = pd.merge(df_kt_bldg_sales, df_bld_rgst, on='emd_cd', how='left')

MemoryError: Unable to allocate 218. GiB for an array with shape (29211534645,) and data type int64

In [1]:
# null 제거
df = df[~df['area_avg'].isnull()]

# 매출액 / 면적
df['sales_per_area'] = (df['sales'] / df['area_avg'])

# inf 값 제거후 컬럼 정리
df = df[~(df['sales_per_area']==np.inf)]
df = df[['pnu','emd_cd','sales_per_area','industry_cd']]

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

KeyError: 'emd_cd'

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

In [ ]:
df['sales_per_area'].median()

In [ ]:
df.describe()

###### ver3

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 면적, 매출액
sql = '''
select t1.pnu, t1.emd_cd, t1.ftc_cate2_cd, t1.sales, t2.gross_area
from (
	select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00' as emd_cd
	  from m1.kt_bldg_sales
	 where base_ym = '202209'
	)as t1
left join (
	select bd_mgt_sn as pnu, emd_cd||'00' as emd_cd, gross_area
	  from m1.kt_bldg_base
	 where base_ym = '202209'
	   and bdtyp_nm not like '%주택%'
	) as t2
  on t1.emd_cd = t2.emd_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','emd_cd','industry_cd','sales','gross_area']


# 매출액 / 면적
df['sales_per_area'] = (df['sales'] / df['gross_area'])

# inf 값 제거후 컬럼 정리
df = df[~(df['sales_per_area']==np.inf)]
df = df[['pnu','emd_cd','sales_per_area','industry_cd']]

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

###### ver4 (나눠서 돌리기)

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from shapely.wkb import loads
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.2f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# emd_cd 가져오기
sql = 'select distinct emd_cd from m1.pnu;'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()
df_emd_cd = pd.DataFrame(rows)
df_emd_cd.columns = ['emd_cd']

df = pd.DataFrame() # 최종 데이터 프레임 생성

for i in df_emd_cd['emd_cd']:
    
    # pnu별 상업용 부동산 면적당 업종별 추정 매출액
    sql = f'''
    with date as (
        select '202209' as base_ym
    )
    select substring(A.bd_mgt_sn, 1, 19) as pnu, B.emd_cd, A.ftc_cate2_cd, B.poly
         , case 
            when A.pred_slng_amt > 0 and C.tot_area > 0 then round(A.pred_slng_amt/C.tot_area, 2)
            else 0 -- 면적이나 매출액 중 한 개라도 null인 경우 0으로 처리
         end as sales_per_area
    from ( -- 건물당 매출액, 업종코드 
        select *
        from m1.kt_bldg_sales 
        where base_ym = (select base_ym from date)
          and pred_slng_amt notnull 
          and pred_slng_amt > 0
        ) A
    join m1.pnu B -- 반경 계산할 폴리곤 조인
        on substring(A.bd_mgt_sn, 1, 19) = B.pnu
    join (-- 연면적 조인	
        select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, sum(tot_area) as tot_area
        from m1.bld_rgst
        where base_ym = (select base_ym from date)
          and tot_area notnull
          and tot_area > 0
        group by 1
    ) C
        on substring(A.bd_mgt_sn, 1, 19) = C.pnu
    where B.emd_cd = {i}
    '''
    
    # 데이터 가져오기
    aws_cursor.execute(sql)
    aws_conn.commit()
    rows = aws_cursor.fetchall()
    
    # polygon 변환
    rows1 = []
    for k in range(len(rows)):
        rows1.append((rows[k][:4]) + (loads(rows[k][4], hex=True).wkt,))
   
    # 최종 데이터 프레임에 병합
    df_sub = pd.DataFrame(rows1)
    df = df.append(df_sub)
    
df.columns = ['pnu','emd_cd','industry_cd','sales_per_area','poly']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = "(" + ("%s," * len(df.columns))[:-4] + "ST_GeomFromText(%s, 4326, 'axis-order=long-lat'))"

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

###### ver5 (final)

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from shapely.wkb import loads
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 면적당 업종별 추정 매출액
sql = '''
with date as (
    select '202209' as base_ym
)
select substring(A.bd_mgt_sn, 1, 19) as pnu, B.emd_cd, A.ftc_cate2_cd, round(A.pred_slng_amt/C.tot_area, 2) as sales_per_area
from ( -- 건물당 매출액, 업종코드 
	select *
	from m1.kt_bldg_sales 
	where base_ym = (select base_ym from date)
	  and pred_slng_amt notnull 
	  and pred_slng_amt > 0
	) A
join m1.pnu B -- 실제 pnu여부 확인
	on substring(A.bd_mgt_sn, 1, 19) = B.pnu
join (-- 연면적 
	select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, sum(tot_area) as tot_area
	from m1.bld_rgst
	where base_ym = (select base_ym from date)
	  and tot_area notnull
	  and tot_area > 0
	group by 1
) C
	on substring(A.bd_mgt_sn, 1, 19) = C.pnu
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

# polygon 변환
rows1 = []
for k in range(len(rows)):
    rows1.append((rows[k][:4]) + (loads(rows[k][4], hex=True).wkt,))

# 데이터프레임화
df = pd.DataFrame(rows1)
df.columns = ['pnu','emd_cd','industry_cd','sales_per_area','poly']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = "(" + ("%s," * len(df.columns))[:-4] + "ST_GeomFromText(%s, 4326, 'axis-order=long-lat'))"

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

###### ver6

In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from shapely.wkb import loads
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.0f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 상업용 부동산 면적당 업종별 추정 매출액
sql = '''
with date as (
    select '202209' as base_ym
)
select substring(A.bd_mgt_sn, 1, 19) as pnu, B.emd_cd, A.ftc_cate2_cd, round(A.pred_slng_amt/C.tot_area, 0) as sales_per_area
from ( -- 건물당 매출액, 업종코드 
	select *
	from m1.kt_bldg_sales 
	where base_ym = (select base_ym from date)
	  and pred_slng_amt notnull 
	  and pred_slng_amt > 0
	) A
join m1.pnu B -- 실제 pnu 여부 확인
	on substring(A.bd_mgt_sn, 1, 19) = B.pnu
join (-- 실제 건물 여부 확인
	select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, sum(tot_area) as tot_area
	from m1.bld_rgst
	where base_ym = (select base_ym from date)
	  and tot_area notnull
	  and tot_area > 0
	group by 1
) C
	on substring(A.bd_mgt_sn, 1, 19) = C.pnu
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

# 데이터프레임화
df = pd.DataFrame(rows)
df.columns = ['pnu','emd_cd','industry_cd','sales_per_area']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

In [2]:
df

,pnu,emd_cd,industry_cd,sales_per_area,create_at,update_at,work_user
0,3114010100102820002,3114010100,B14,18596,2022-11-17,2022-11-17,du.Park
1,3114010100102820002,3114010100,B21,323939,2022-11-17,2022-11-17,du.Park
2,3114010100102820002,3114010100,B20,4287,2022-11-17,2022-11-17,du.Park
3,3114010100102820002,3114010100,A07,41103,2022-11-17,2022-11-17,du.Park
4,3114010100102820002,3114010100,A02,28577,2022-11-17,2022-11-17,du.Park
...,...,...,...,...,...,...,...
48382391,5013032026117900000,5013032026,C01,56667,2022-11-17,2022-11-17,du.Park
48382392,5013032026117900000,5013032026,B10,35184,2022-11-17,2022-11-17,du.Park
48382393,5013032026117900000,5013032026,A05,9922,2022-11-17,2022-11-17,du.Park
48382394,5013032026117900000,5013032026,A10,107387,2022-11-17,2022-11-17,du.Park


In [3]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-17 11:58:13.720618  :  0
2022-11-17 11:58:18.847739  :  10000
2022-11-17 11:58:21.255784  :  20000
2022-11-17 11:58:23.367939  :  30000
2022-11-17 11:58:25.096620  :  40000
2022-11-17 11:58:26.829536  :  50000
2022-11-17 11:58:28.395832  :  60000
2022-11-17 11:58:30.099884  :  70000
2022-11-17 11:58:31.772255  :  80000
2022-11-17 11:58:33.629861  :  90000
2022-11-17 11:58:35.274822  :  100000
2022-11-17 11:58:36.935789  :  110000
2022-11-17 11:58:38.578750  :  120000
2022-11-17 11:58:40.259649  :  130000
2022-11-17 11:58:41.942879  :  140000
2022-11-17 11:58:44.147351  :  150000
2022-11-17 11:58:45.781923  :  160000
2022-11-17 11:58:47.484332  :  170000
2022-11-17 11:58:50.042087  :  180000
2022-11-17 11:58:51.801522  :  190000
2022-11-17 11:58:53.506677  :  200000
2022-11-17 11:58:55.143426  :  210000
2022-11-17 11:58:56.881886  :  220000
2022-11-17 11:58:58.556909  :  230000
2022-11-17 11:59:00.534801  :  240000
2022-11-17 11:59:02.852006  :  250000
2022-11-17 11:59:04.499075